# Consinstently estimating Markov Chains with Noisy Aggregated Data

## Notebook con esperimenti numerici per il seminario di fine corso di Metodi Numerici per le Catene di Markov (versione stazionaria)

In [1]:
# Import some basic stuff
import numpy as np
import scipy
import matplotlib.pyplot as plt
from utilities import P_mom_stationary, add_noise, generate_random_P

In [2]:
# Number of repeated observations
K = 30
# Population size
N = 100
# Number of states
S = 20
# Number of timesteps
T = 2000

### Generate the data

Compared to the non-stationary case, we now have to compute the steady state vector $\pi$ of the Markov chain. This is the thing in the seminar that is the most related to the course.

Since the size of the matrix $P$ is modest, I go for a direct method: LU factorization with the GTH trick

In [3]:
def compute_stationary_LU_rough(P:np.ndarray) -> tuple[np.ndarray, float, float]:
    #let's start very roughly: solve (I-P.T)@pi = 0 via LU factorization
    S = P.shape[0]
    A = np.eye(S)-P.T
    M,L,U = scipy.linalg.lu(A)
    LU_error = np.linalg.norm(M@L@U - A)
    pi = np.zeros(S)
    pi[-1] = 1
    for i in range(S-2,-1,-1):
        pi[i] = (-U[i,i+1:]@pi[i+1:])/U[i,i]
    pi /= pi.sum()
    res = np.linalg.norm(A@pi)
    return pi, res, LU_error

#TODO
#def compute_stationary_LU_GTH(P:np.ndarray) -> np.ndarray:
#    S = P.shape[0]

In [4]:
# True transition matrix
P = generate_random_P(S,'dirichlet',precision=0.5)

# Initial distribution
pi_rough, res, _ = compute_stationary_LU_rough(P)
print(f'The norm of (I-P.T)@pi is {res}')

The norm of (I-P.T)@pi is 1.3218586933369176e-16


Let's generate the observed data. We immediately generate the $K$ observations.

This is done by generating $n_t\sim\mathrm{Multinomial}(N,\pi)$ for each $t\in[T]$.

In [5]:
pi_0 = pi_rough

In [6]:
mu_t = pi_0.T
n_t_vector = []
y_t_vector = []
for t in range(T):
    # create K observations of the observed data 
    # (multinomial draw from the marginal distribution)
    n_t = np.random.multinomial(n=N, pvals=mu_t, size=K)
    # create noisy observations
    y_t, _ = add_noise(n_t)
    # append the observations
    n_t_vector.append(n_t)
    y_t_vector.append(y_t)
    # In the stationary case, the marginal distribution is equal 
    # to the stationary distribution

**OSS**: `n_t_vector` and `y_t_vector` are lists of length $T$, the item in the list in position $t\in\{0,\dots,T-1\}$ is a $K\times S$ `np.ndarray` that contains the $K$ observations for timestep $t+1$ 

### Estimators of $P$

In [7]:
y_array = np.array(y_t_vector)
A = np.eye(S)

In [8]:
P_mom = P_mom_stationary(y_array=y_array, A=A, N=N)
print(f'P_mom.shape = {P_mom.shape}')
print(f'The 2-norm of the difference between P and P_mom is {np.linalg.norm(P-P_mom)}')

P_mom.shape = (20, 20)
The 2-norm of the difference between P and P_mom is 3.5870520250972224


Let's move on to the Conditional Least Squares estimator

In [9]:
X = y_array[:-1].transpose(1,0,2)
Y = y_array[1:].transpose(1,0,2)
print(f'X.shape = {X.shape},\t (K, T-1, S) = ({K}, {T-1}, {S})')

X.shape = (30, 1999, 20),	 (K, T-1, S) = (30, 1999, 20)


I feel like we should be taking an average over the $K$ trials, for fairness

In [19]:
coll_P_cls = np.array([np.linalg.lstsq(X[k],Y[k])[0] for k in range(K-1)])
P_CLS = coll_P_cls.mean(axis=0)
P_CLS.shape

/tmp/ipykernel_13115/1739891240.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coll_P_cls = np.array([np.linalg.lstsq(X[k],Y[k])[0] for k in range(K-1)])


In [25]:
np.linalg.norm(P_CLS-P)

3.5937108366452972